In [1]:
import pandas as pd
import numpy as np

# Settings

In [2]:
# input dataset
series_csv = "../../HERON/20210101_to_20211014/EVB-P1941076_Heron_Parking.csv"
# series_csv = "../../HERON/20210101_to_20211014/EVB-P1941076_Heron_Parking.csv"
# Choose output file frequency / resolution in minutes
resolution = 60

# Output name
output_csv = f"{series_csv.split('/')[-1].split('.')[0]}_mean_demand_20210102_to_20211013_{str(resolution)+'min'}.csv"

# Read csv

In [3]:
ts = pd.read_csv(series_csv, 
                delimiter=',', 
                header=0, 
                index_col=None, 
                parse_dates=False)
ts.head()

,Connector ID,Card ID,Start date,End date,Duration,kWh,Charging cost (EUR),Transaction fee (EUR),Reimbursable (EUR),Charging Type
0,2001339,NL-EVB-C00329175-5,1/8/2021 12:56,1/8/2021 17:28,4:32:09 AM,6.93,0,0,1.39,Private
1,2001339,NL-EVB-C00329175-5,1/11/2021 9:11,1/11/2021 19:25,10:14:43 AM,6.75,0,0,1.35,Private
2,2001339,NL-EVB-C00329175-5,1/12/2021 8:43,1/12/2021 19:28,10:45:20 AM,4.00,0,0,0.80,Private
3,2001339,NL-EVB-C00329175-5,1/13/2021 8:42,1/13/2021 17:12,8:29:30 AM,4.26,0,0,0.85,Private
4,1950779,NL-EVB-C00329175-5,1/13/2021 8:49,1/13/2021 19:17,10:27:50 AM,9.49,0,0,1.90,Private


# Drop useless columns and create useful ones
Also drop rows with very small duration

In [4]:
ts = ts.drop(columns=["Card ID",
                      "Charging cost (EUR)",
                      "Transaction fee (EUR)",
                      "Reimbursable (EUR)",
                      "Charging Type"])

ts["Start date"] = pd.to_datetime(ts['Start date'])
ts["End date"] = pd.to_datetime(ts['End date'])
ts["Duration (h)"] = (ts["End date"] - ts["Start date"]) / \
    np.timedelta64(1, 's') / 3600

# drop small durations
ts = ts[ts["Duration (h)"] > 5/60]

ts = ts.drop(columns=["Duration"])
ts["Mean Demand (kw)"] = ts["kWh"] / ts["Duration (h)"]
ts.head()

,Connector ID,Start date,End date,kWh,Duration (h),Mean Demand (kw)
0,2001339,2021-01-08 12:56:00,2021-01-08 17:28:00,6.93,4.533333,1.528676
1,2001339,2021-01-11 09:11:00,2021-01-11 19:25:00,6.75,10.233333,0.659609
2,2001339,2021-01-12 08:43:00,2021-01-12 19:28:00,4.00,10.750000,0.372093
3,2001339,2021-01-13 08:42:00,2021-01-13 17:12:00,4.26,8.500000,0.501176
4,1950779,2021-01-13 08:49:00,2021-01-13 19:17:00,9.49,10.466667,0.906688


# Create in / out power and total power dataset

## in (plug) / out (unplug) power datasets

In [5]:
# this dataframe contains the plug in times along with the differential contribution of the plug to the station demand
plug = pd.DataFrame(data=ts[["Start date", "Mean Demand (kw)"]])
plug.rename(columns={"Mean Demand (kw)": "Diff Demand (kw)"}, inplace=True)
plug = plug.set_index("Start date")

# this dataframe contains the plug out times along with the differential contribution (<0) of the unplugging process to the station demand
unplug = pd.DataFrame(data=ts[["End date", "Mean Demand (kw)"]])
unplug.rename(columns={"Mean Demand (kw)": "Diff Demand (kw)"}, inplace=True)
unplug["Diff Demand (kw)"] = - unplug["Diff Demand (kw)"]
unplug = unplug.set_index("End date")
unplug.head(20)


,Diff Demand (kw)
End date,
2021-01-08 17:28:00,-1.528676
2021-01-11 19:25:00,-0.659609
2021-01-12 19:28:00,-0.372093
2021-01-13 17:12:00,-0.501176
2021-01-13 19:17:00,-0.906688
2021-01-14 18:41:00,-0.817550
2021-01-14 18:45:00,-0.455241
2021-01-15 19:33:00,-1.306410
2021-01-15 20:59:00,-1.238430


## Regularize time scales and normalize middle values
Need to get rid of irregular indices to obtain a timeseries of the selected resolution
### Create regular time index

In [18]:
# in
periodic_index = pd.date_range(
    start=plug.index[0].date(), end=plug.index[-1].date(), freq=str(resolution)+'min')

regular_index_plug = pd.DataFrame(index=periodic_index, columns=["Diff Demand (kw)"])

# out
periodic_index = pd.date_range(
    start=unplug.index[0].date(), end=unplug.index[-1].date(), freq=str(resolution)+'min')

regular_index_unplug = pd.DataFrame(
    index=periodic_index, columns=["Diff Demand (kw)"])
regular_index_unplug.head()


,Diff Demand (kw)
2021-01-08 00:00:00,NaN
2021-01-08 01:00:00,NaN
2021-01-08 02:00:00,NaN
2021-01-08 03:00:00,NaN
2021-01-08 04:00:00,NaN


### Merge scales
Both datasets are merged with the regular indices. Cumsums are important to get a mean demand contribution at each timestep for each dataset.

In [19]:
# in
plug_aligned = pd.concat([plug, regular_index_plug],
                         join='outer').sort_index().fillna(0)
plug_aligned["inDemand (kw)"] = plug_aligned["Diff Demand (kw)"].cumsum()
plug_aligned.drop(columns=["Diff Demand (kw)"], inplace=True)
plug_aligned = plug_aligned[~plug_aligned.index.duplicated(keep='first')]

# out
unplug_aligned = pd.concat([unplug, regular_index_unplug],
                         join='outer').sort_index().fillna(0)
unplug_aligned["outDemand (kw)"] = unplug_aligned["Diff Demand (kw)"].cumsum()
unplug_aligned.drop(columns=["Diff Demand (kw)"], inplace=True)
unplug_aligned = unplug_aligned[~unplug_aligned.index.duplicated(keep='first')]
unplug_aligned.head(30)

,outDemand (kw)
2021-01-08 00:00:00,0.000000
2021-01-08 01:00:00,0.000000
2021-01-08 02:00:00,0.000000
2021-01-08 03:00:00,0.000000
2021-01-08 04:00:00,0.000000
2021-01-08 05:00:00,0.000000
2021-01-08 06:00:00,0.000000
2021-01-08 07:00:00,0.000000
2021-01-08 08:00:00,0.000000
2021-01-08 09:00:00,0.000000


### Move to regular scale, normalizing mid values
Then both datasets are converted to the regular index scale through normalization. irregular indices are dropped afterwards.

In [20]:
# in
for index, row in plug_aligned.iterrows():
    if index.minute != 0:  # track irregular indices
        location = plug_aligned.index.get_loc(index)  # keep location as int
        # calculate normalized value proportionally
        normalized_value = plug_aligned.loc[index] * (1- index.minute / resolution)
        plug_aligned.drop(index, inplace=True)  # drop irregular index
        # assign normalized value to the previous regular index
        plug_aligned.iloc[location-1] = normalized_value

# out
for index, row in unplug_aligned.iterrows():
    if index.minute != 0:  # track irregular indices
        location = unplug_aligned.index.get_loc(index)  # keep location as int
        # calculate normalized value proportionally
        normalized_value = unplug_aligned.loc[index] * (1 + index.minute / resolution)
        normalized_value = - plug_aligned.loc[index]
        unplug_aligned.drop(index, inplace=True)  # drop irregular index
        # assign normalized value to the previous regular index
        unplug_aligned.iloc[location-1] = normalized_value
plug_aligned.head(30)


,inDemand (kw)
2021-01-08 00:00:00,0.000000
2021-01-08 01:00:00,0.000000
2021-01-08 02:00:00,0.000000
2021-01-08 03:00:00,0.000000
2021-01-08 04:00:00,0.000000
2021-01-08 05:00:00,0.000000
2021-01-08 06:00:00,0.000000
2021-01-08 07:00:00,0.000000
2021-01-08 08:00:00,0.000000
2021-01-08 09:00:00,0.000000


## Create total power dataset by summing up contributions of plugs and unplugs at each timestep

In [21]:
# demand = plug_aligned.add(unplug_aligned, fill_value=0)
demand = pd.concat([plug_aligned, unplug_aligned], axis=1).fillna(0)
demand["Total Demand (kw)"] = demand["inDemand (kw)"] + demand["outDemand (kw)"]
demand[abs(demand["Total Demand (kw)"]) < 1e-3] = 0
demand.head(50)

,inDemand (kw),outDemand (kw),Total Demand (kw)
2021-01-08 00:00:00,0.000000,0.000000,0.000000
2021-01-08 01:00:00,0.000000,0.000000,0.000000
2021-01-08 02:00:00,0.000000,0.000000,0.000000
2021-01-08 03:00:00,0.000000,0.000000,0.000000
2021-01-08 04:00:00,0.000000,0.000000,0.000000
2021-01-08 05:00:00,0.000000,0.000000,0.000000
2021-01-08 06:00:00,0.000000,0.000000,0.000000
2021-01-08 07:00:00,0.000000,0.000000,0.000000
2021-01-08 08:00:00,0.000000,0.000000,0.000000
2021-01-08 09:00:00,0.000000,0.000000,0.000000


In [10]:
demand.to_csv(output_csv)